# Table of Contents
 <p><div class="lev1"><a href="#Summary"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></div><div class="lev1"><a href="#Imports"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></div><div class="lev1"><a href="#Download"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download</a></div><div class="lev1"><a href="#Load-data"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load data</a></div><div class="lev1"><a href="#Save-to-database"><span class="toc-item-num">5&nbsp;&nbsp;</span>Save to database</a></div>

# Summary


https://genomeinterpretation.org/content/4-SUMO_ligase

----

# Imports

In [1]:
%run imports.ipynb

2016-08-25 22:15:57.256543


In [2]:
%run db.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-08-25 22:15:57.450829


MySQL database already running...


In [3]:
NOTEBOOK_NAME = 'cagi4_sumo_ligase'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

# Download

Downlaod `4-SUMO_ligase_answer_key.txt`, `4-SUMO_ligase_predictions.zip`, and `4-SUMO_ligase_dataset.txt` from https://genomeinterpretation.org/content/4-SUMO_ligase.

In [4]:
!ls $NOTEBOOK_NAME

4-SUMO_ligase_answer_key.txt   Group_39  Group_42  Group_45
4-SUMO_ligase_dataset.txt      Group_40  Group_43  Group_46
4-SUMO_ligase_predictions.zip  Group_41  Group_44  Group_47


# Load data

In [5]:
!7z x -bd -aos "$NOTEBOOK_NAME/4-SUMO_ligase_predictions.zip" -o"$NOTEBOOK_NAME"


7-Zip [64] 15.09 beta : Copyright (c) 1999-2015 Igor Pavlov : 2015-10-16
p7zip Version 15.09 beta (locale=en_CA.UTF-8,Utf16=on,HugeFiles=on,64 bits,64 CPUs x64)

Scanning the drive for archives:
1 file, 3097916 bytes (3026 KiB)

Extracting archive: cagi4_sumo_ligase/4-SUMO_ligase_predictions.zip
--
Path = cagi4_sumo_ligase/4-SUMO_ligase_predictions.zip
Type = zip
Physical Size = 3097916

Everything is Ok

Files: 0
Size:       0
Compressed: 3097916


In [6]:
!ls $NOTEBOOK_NAME

4-SUMO_ligase_answer_key.txt   Group_39  Group_42  Group_45
4-SUMO_ligase_dataset.txt      Group_40  Group_43  Group_46
4-SUMO_ligase_predictions.zip  Group_41  Group_44  Group_47


In [7]:
DF = pd.read_csv(op.join(NOTEBOOK_NAME, '4-SUMO_ligase_answer_key.txt'), sep='\t')

In [8]:
DF.head()

,AA_substitution(s),competitive_growth_score,stderr
0,#Subset 1,NaN,NaN
1,A10G,0.545179,0.276260
2,A129V,-0.106306,0.052569
3,A131S,0.971831,0.327676
4,A150T,0.068733,0.265307


In [9]:
DF = DF[DF['AA_substitution(s)'] != '#Subset 1']

In [10]:
DF.head()

,AA_substitution(s),competitive_growth_score,stderr
1,A10G,0.545179,0.276260
2,A129V,-0.106306,0.052569
3,A131S,0.971831,0.327676
4,A150T,0.068733,0.265307
5,A150W,0.451040,0.095980


In [11]:
DF.tail()

,AA_substitution(s),competitive_growth_score,stderr
5107,"Y87H,P105E,A133P",-0.142567,0.129594
5108,"Y87N,A106T,I107N,T108A",-0.241470,0.119112
5109,"Y87N,E98G",0.135409,0.137413
5110,"Y87N,E98K",1.998385,0.151798
5111,"Y87N,N124R",-0.240334,0.107067


In [12]:
DF['uniprot_id'] = 'P63279'
DF['partner_uniprot_id'] = np.nan
DF['uniprot_mutation'] = DF['AA_substitution(s)'].str.replace(',', '.')
DF['del_score_exp'] = DF['competitive_growth_score']

In [13]:
DF.head()

,AA_substitution(s),competitive_growth_score,stderr,uniprot_id,partner_uniprot_id,uniprot_mutation,del_score_exp
1,A10G,0.545179,0.276260,P63279,NaN,A10G,0.545179
2,A129V,-0.106306,0.052569,P63279,NaN,A129V,-0.106306
3,A131S,0.971831,0.327676,P63279,NaN,A131S,0.971831
4,A150T,0.068733,0.265307,P63279,NaN,A150T,0.068733
5,A150W,0.451040,0.095980,P63279,NaN,A150W,0.451040


In [14]:
DF1_bak = DF.copy()

# Save to database

In [15]:
DF = DF1_bak.copy()

In [16]:
DF.head()

,AA_substitution(s),competitive_growth_score,stderr,uniprot_id,partner_uniprot_id,uniprot_mutation,del_score_exp
1,A10G,0.545179,0.276260,P63279,NaN,A10G,0.545179
2,A129V,-0.106306,0.052569,P63279,NaN,A129V,-0.106306
3,A131S,0.971831,0.327676,P63279,NaN,A131S,0.971831
4,A150T,0.068733,0.265307,P63279,NaN,A150T,0.068733
5,A150W,0.451040,0.095980,P63279,NaN,A150W,0.451040


In [17]:
columns = [
    'uniprot_id', 'partner_uniprot_id', 'uniprot_mutation', 'del_score_exp',
]

In [18]:
print(DF.shape[0])
print(DF[columns].drop_duplicates().shape[0])

5111
5111


In [19]:
t = db.import_df(DF[columns], NOTEBOOK_NAME)

In [20]:
t.create_indexes([
    (['uniprot_id', 'uniprot_mutation'], True),
])

In [21]:
t.add_idx_column()

5111

In [22]:
t.compress()

File size before: 0.22 MB
File size after: 0.11 MB
File size savings: 0.11 MB (50.57 %)


(CompletedProcess(args=['myisampack', '--no-defaults', '/home/kimlab1/database_data/biodb/recipes/mutation_sets/notebooks/mysqld/mutation_sets/cagi4_sumo_ligase.MYI'], returncode=0, stdout='Remember to run myisamchk -rq on compressed tables\n', stderr=''),
 CompletedProcess(args=['myisamchk', '-rq', '/home/kimlab1/database_data/biodb/recipes/mutation_sets/notebooks/mysqld/mutation_sets/cagi4_sumo_ligase.MYI'], returncode=0, stdout="- check record delete-chain\n- recovering (with sort) MyISAM-table '/home/kimlab1/database_data/biodb/recipes/mutation_sets/notebooks/mysqld/mutation_sets/cagi4_sumo_ligase.MYI'\nData records: 5111\n- Fixing index 1\n- Fixing index 2\n", stderr=''))

In [23]:
print(datetime.datetime.now())

2016-08-25 22:16:03.322636
